In [ ]:
# Learning with Group Privacy
from MLModel import *

from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset

import torch
import numpy as np
import os

import time

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

In [ ]:
scattering, K, (h, w) = get_scatter_transform(dataset="cifar10")
scattering.to(device)

def get_scattered_feature(dataset):
    scatters = []
    targets = []
    
    loader = torch.utils.data.DataLoader(
        dataset, batch_size=256, shuffle=True, num_workers=1, pin_memory=True)

    
    for (data, target) in loader:
        data, target = data.to(device), target.to(device)
        if scattering is not None:
            data = scattering(data)
        scatters.append(data)
        targets.append(target)

    scatters = torch.cat(scatters, axis=0)
    targets = torch.cat(targets, axis=0)

    data = torch.utils.data.TensorDataset(scatters, targets)
    return data


def load_cifar10():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    train_transforms = [transforms.ToTensor(), normalize,]

    train = datasets.CIFAR10(root="~/data/", train=True,
                                 transform=transforms.Compose(train_transforms),
                                 download=True)

    test = datasets.CIFAR10(root="~/data/", train=False,
                                transform=transforms.Compose(
                                    [transforms.ToTensor(), normalize]
                                ))
    
    # get scattered features
    train = get_scattered_feature(train)
    test = get_scattered_feature(test)
    
    train_data = train[:][0].squeeze().cpu().float()
    train_label = train[:][1].cpu()
    
    test_data = test[:][0].squeeze().cpu().float()
    test_label = test[:][1].cpu()

    dataset = [(train_data, train_label), (test_data, test_label)]
    return dataset

In [ ]:
d = load_cifar10()

train_data = torch.tensor(d[0][0]).to(device)
train_label = torch.tensor(d[0][1]).to(device)

test_data = torch.tensor(d[1][0]).to(device)
test_label = torch.tensor(d[1][1]).to(device)

data_size = len(train_label)
print("#. training records=",data_size)

print("shape of training records: ",d[0][0].shape)

torch.cuda.empty_cache()

In [ ]:
"""
FL model parameters.
"""
import warnings
warnings.filterwarnings("ignore")

lr = 1.5
iters = 500

configs = {
    'output_size': 10,
    'data_size': data_size,
    'model': 'scatter',
    'data': d,
    'lr': lr,
    'E': iters,
    'delta': 1e-5,
    'q': 0.1,
    'clip': 0.1,
    'batch_size': 128,
    'device': device
}

In [ ]:
def test_acc(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        tot_sample = len(test_data)
        #for i in range(len(test_data)):

        t_pred_y = model(test_data)
        _, predicted = torch.max(t_pred_y, 1)
        correct += (predicted == test_label).sum().item()

        acc = correct / tot_sample
    model.train()
    return acc

def train():
    model.train()
    criterion = nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.SGD(model.parameters(), lr=configs['lr'], momentum=0.9)
    # optimizer = torch.optim.Adam(model.parameters())

    start_time = time.time()
    for e in range(configs['E']):
        model.train()
        # randomly select q fraction samples from dataset by poisson sampling
        idx = np.where(np.random.rand(len(torch_train[:][0])) < configs['q'])[0]

        sampled_dataset = TensorDataset(torch_train[idx][0], torch_train[idx][1])
        sample_data_loader = DataLoader(
            dataset=sampled_dataset,
            batch_size=configs['batch_size'],
            shuffle=True
        )

        optimizer.zero_grad()

        clipped_grads = {name: torch.zeros_like(param) for name, param in model.named_parameters()}
        for batch_x, batch_y in sample_data_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            pred_y = model(batch_x.float())
            loss = criterion(pred_y, batch_y.long())

            # bound l2 sensitivity (gradient clipping)
            # clip each of the gradient in subset
            for i in range(loss.size()[0]):
                loss[i].backward(retain_graph=True)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=configs['clip'])
                for name, param in model.named_parameters():
                    clipped_grads[name] += param.grad 
                model.zero_grad()

        # add Gaussian noise
        for name, param in model.named_parameters():
            clipped_grads[name] += gaussian_noise(clipped_grads[name].shape, configs['clip'], configs['sigma'], device=device)

        # scale back
        for name, param in model.named_parameters():
            clipped_grads[name] /= (configs['data_size']*configs['q'])

        for name, param in model.named_parameters():
            param.grad = clipped_grads[name]

        # update local model
        optimizer.step()
        
        if (e+1)%2 == 0:
            acc = test_acc(model)
            print("iters = {:d}, acc = {:.4f}".format(e+1, acc), " Time taken: %.2fs" % (time.time() - start_time))
    return acc

# Varying group size $m$

In [ ]:
"""
noise scale is calibrated using binary search,
see 'calibrate_sgm_noise.ipynb' file
"""
m_list = [8, 16, 32, 64]
m_list_ours = [8, 16, 24, 32, 40, 48, 56, 64]

noise_rdp= [38.10931396484375, 93.24703979492188, 228.32774353027344, 559.2231292724609]
noise_ours= [30.653938055038452, 52.265591621398926, 73.35555791854858, 94.27658557891846, 115.12147188186646, 135.92471837997437, 156.70357584953308, 177.46637225151062]

### Naive RDP

In [ ]:
for i, sigma in enumerate(noise_rdp):
    print("sigma =",sigma)
    configs['sigma'] = sigma
    acc_list = []
    for _ in range(5):
        model = ScatterLinear(243, (8, 8), input_norm="GroupNorm", num_groups=27).to(device)
        torch_train = TensorDataset(torch.tensor(train_data), torch.tensor(train_label))
        acc = train()
        acc_list.append(acc)
    print("m = {:d}, avg acc = {:.4f}".format(m_list[i], np.mean(acc_list)) )
    print("=========")

### Ours

In [ ]:
for sigma in noise_ours:
    print("sigma =",sigma)
    configs['sigma'] = sigma
    acc_list = []
    for _ in range(5):
        model = ScatterLinear(243, (8, 8), input_norm="GroupNorm", num_groups=27).to(device)
        torch_train = TensorDataset(torch.tensor(train_data), torch.tensor(train_label))
        acc = train()
        acc_list.append(acc)
    print("avg acc =", np.mean(acc_list))
    print("=========")

# Vary privacy parameter $\epsilon$

In [ ]:
"""
noise scale is calibrated using binary search,
see 'calibrate_sgm_noise.ipynb' file
"""
# m = 32
eps_list = [8, 7, 6, 5, 4, 3, 2]

noise_rdp= [62.96098327636719, 70.53829956054688, 80.5775146484375, 94.09147644042969, 114.18656921386719, 147.5915069580078, 211.9739990234375]
noise_ours= [29.204277992248535, 32.63718247413635, 37.39244818687439, 43.51409435272217, 52.82178044319153, 68.23326706886292, 97.94802665710449]

## Naive RDP

In [ ]:
for sigma in noise_rdp:
    print("sigma =",sigma)
    configs['sigma'] = sigma
    acc_list = []
    for _ in range(5):
        model = ScatterLinear(243, (8, 8), input_norm="GroupNorm", num_groups=27).to(device)
        torch_train = TensorDataset(torch.tensor(train_data), torch.tensor(train_label))
        acc = train()
        acc_list.append(acc)
    print("avg acc =", np.mean(acc_list))
    print("=========")

## Ours

In [ ]:
for sigma in noise_ours:
    print("sigma =",sigma)
    configs['sigma'] = sigma
    acc_list = []
    for _ in range(5):
        model = ScatterLinear(243, (8, 8), input_norm="GroupNorm", num_groups=27).to(device)
        torch_train = TensorDataset(torch.tensor(train_data), torch.tensor(train_label))
        acc = train()
        acc_list.append(acc)
    print("avg acc =", np.mean(acc_list))
    print("=========")